In [1]:
from __future__ import division
import os
import numpy as np
import random
from scipy import pi
from itertools import islice

In [2]:
images_path = "/Users/yesas/Desktop/Dissertation/driving_dataset/"
Steer_angle = "/Users/yesas/Desktop/Dissertation/driving_dataset/data.txt"

images = []
Angles = []


In [3]:
with open(Steer_angle) as f:
    for line in f:
        image_name, angle = line.split()
        
        image_path = os.path.join(images_path, image_name)
        images.append(image_path)
        
        angle_radians = float(angle) * (pi / 180) 
        Angles.append(angle_radians)
Angles = np.array(Angles)
print(str(len(images))+" "+str(len(Angles)))

45406 45406


In [4]:
Train_test_split = int(len(images) * 0.8)

train_x = images[:Train_test_split]
train_y = Angles[:Train_test_split]

test_x = images[Train_test_split:]
test_y = Angles[Train_test_split:]

print("Total Input Images in Train set :", len(train_x))
print("Total Output Values in Train set :", len(train_y))
print("Total Input Images in Test set :", len(test_x))
print("Total Output Values in Train set :", len(test_y))

Total Input Images in Train set : 36324
Total Output Values in Train set : 36324
Total Input Images in Test set : 9082
Total Output Values in Train set : 9082


In [5]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

sns.distplot(train_y, bins = 50)
sns.distplot(test_y,bins = 50)

plt.title("Steering Wheel angle in train and test")
plt.xlabel("Angle")
plt.ylabel("Bin count")
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



<Figure size 640x480 with 1 Axes>

In [6]:
#Model 0: Base line Model: y test pred = mean(y_train_i) 

train_mean_y = np.mean(train_y)

print('Test MSE(MEAN):%f' % np.mean(np.square(test_y-train_mean_y)) )

print('Test_MSE (ZERO):%f' % np.mean(np.square(test_y-0.0)))

Test MSE(MEAN):0.191127
Test_MSE (ZERO):0.190891


In [7]:
train_batch_pointer = 0
test_batch_pointer = 0
def loadTrainBatch(batch_size):
    global train_batch_pointer
    x_result = []
    y_result = []
    for i in range(batch_size):
        read_image = cv2.imread(train_x[(train_batch_pointer + i) % len(train_x)])
        
        read_image_road = read_image[-150:] 
        read_image_resize = cv2.resize(read_image_road, (200, 66)) 
        read_image_final = read_image_resize/255.0  
        x_result.append(read_image_final) 
        y_result.append(train_y[(train_batch_pointer + i) % len(train_y)])
        
    train_batch_pointer += batch_size
        
    return x_result, y_result

In [8]:
def loadTestBatch(batch_size):
    global test_batch_pointer
    x_result = []
    y_result = []
    for i in range(batch_size):
        read_image = cv2.imread(test_x[(test_batch_pointer + i) % len(test_x)]) 
        read_image_road = read_image[-150:] 
        read_image_resize = cv2.resize(read_image_road, (200, 66)) 
        read_image_final = read_image_resize/255.0  
        
        x_result.append(read_image_final) 
        
        y_result.append(test_y[(test_batch_pointer + i) % len(test_y)]) 
        
    test_batch_pointer += batch_size
        
    return x_result, y_result

In [9]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
conv_filter_shapes = {
    "c1_filter": [5, 5, 3, 24],
    "c2_filter": [5, 5, 24, 36],
    "c3_filter": [5, 5, 36, 48],
    "c4_filter": [3, 3, 48, 64],
    "c5_filter": [3, 3, 64, 64]
}


In [11]:
fc_connection_shapes = {
    "f1_shape": [1*18*64, 1164],
    "f2_shape": [1164, 100],
    "f3_shape": [100, 50],
    "f4_shape": [50,10],
    "f5_shape": [10,1],
    
}

In [12]:
conv_weights = {
    "c1_weights": tf.Variable(tf.truncated_normal(conv_filter_shapes["c1_filter"], stddev=0.1), name="c1_weights"),
    "c2_weights": tf.Variable(tf.truncated_normal(conv_filter_shapes["c2_filter"],stddev=0.1), name="c2_weights"),
    "c3_weights": tf.Variable(tf.truncated_normal(conv_filter_shapes["c3_filter"],stddev=0.1), name="c3_weights"),
    "c4_weights": tf.Variable(tf.truncated_normal(conv_filter_shapes["c4_filter"], stddev=0.1), name="c4_weights"),
    "c5_weights": tf.Variable(tf.truncated_normal(conv_filter_shapes["c5_filter"], stddev=0.1), name="c5_weights"),
    "f1_weights": tf.Variable(tf.truncated_normal(fc_connection_shapes["f1_shape"], stddev=0.1), name="f1_weights"),
    "f2_weights": tf.Variable(tf.truncated_normal(fc_connection_shapes["f2_shape"], stddev=0.1), name="f2_weights"),
    "f3_weights": tf.Variable(tf.truncated_normal(fc_connection_shapes["f3_shape"], stddev=0.1), name="f3_weights"),
    "f4_weights": tf.Variable(tf.truncated_normal(fc_connection_shapes["f4_shape"], stddev=0.1), name="f3_weights"),
    "f5_weights": tf.Variable(tf.truncated_normal(fc_connection_shapes["f5_shape"], stddev=0.1), name="f3_weights")
    
}

In [13]:
conv_biases = {
    "c1_biases": tf.Variable(tf.constant(0.1,shape = [24]), name="c1_biases"),
    "c2_biases": tf.Variable(tf.constant(0.1, shape = [36]), name="c2_biases"), 
    "c3_biases": tf.Variable(tf.constant(0.1, shape = [48]), name="c3_biases"),
    "c4_biases": tf.Variable(tf.constant(0.1, shape = [64]), name="c4_biases"),  
    "c5_biases": tf.Variable(tf.constant(0.1, shape = [64]), name="c5_biases"),
                             
    "f1_biases": tf.Variable(tf.constant(0.1, shape = [1164]), name="f1_biases"),
    "f2_biases": tf.Variable(tf.constant(0.1, shape = [100]), name="f2_biases"),
    "f3_biases": tf.Variable(tf.constant(0.1, shape = [50]), name="f3_biases"),
    "f4_biases": tf.Variable(tf.constant(0.1, shape = [10]), name="f4_biases"),
    "f5_biases": tf.Variable(tf.constant(0.1, shape = [1]), name="f5_biases")
}

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x_input = tf.placeholder(tf.float32, shape = [None, 66, 200, 3], name = "Plc_1")
y_true = tf.placeholder(tf.float32, name = "Plc_2")

input_image = x_input

In [15]:
x_input

<tf.Tensor 'Plc_1:0' shape=(?, 66, 200, 3) dtype=float32>

In [16]:
c_layer_1 = tf.nn.conv2d(input_image, conv_weights["c1_weights"], strides=[1, 2, 2, 1], padding="VALID", name="c_layer_1")
c_layer_1 += conv_biases["c1_biases"]
c_layer_1 = tf.nn.relu(c_layer_1)


c_layer_2 = tf.nn.conv2d(c_layer_1, conv_weights["c2_weights"], strides=[1, 2, 2, 1], padding="VALID", name="c_layer_2")
c_layer_2 += conv_biases["c2_biases"]
c_layer_2 = tf.nn.relu(c_layer_2)


c_layer_3 = tf.nn.conv2d(c_layer_2, conv_weights["c3_weights"], strides=[1, 2, 2, 1], padding="VALID", name="c_layer_3")
c_layer_3 += conv_biases["c3_biases"]
c_layer_3 = tf.nn.relu(c_layer_3)

c_layer_4 = tf.nn.conv2d(c_layer_3, conv_weights["c4_weights"], strides=[1, 1, 1, 1], padding="VALID", name="c_layer_4")
c_layer_4 += conv_biases["c4_biases"]
c_layer_4 = tf.nn.relu(c_layer_4)

c_layer_5 = tf.nn.conv2d(c_layer_4, conv_weights["c5_weights"], strides=[1, 1, 1, 1], padding="VALID", name="c_layer_5")
c_layer_5 += conv_biases["c5_biases"]
c_layer_5 = tf.nn.relu(c_layer_5)

keep_prob = tf.placeholder(tf.float32)

feature_map = tf.reshape(c_layer_5, [-1, 1152])

# Fully Connected Layer 1 | Dropout
fc_layer_1 = tf.nn.relu(tf.matmul(feature_map, conv_weights["f1_weights"]) + conv_biases["f1_biases"])
fc_layer_1_drop = tf.nn.dropout(fc_layer_1, keep_prob)

fc_layer_2 = tf.nn.relu(tf.matmul(fc_layer_1_drop, conv_weights["f2_weights"]) + conv_biases["f2_biases"])
fc_layer_2_drop = tf.nn.dropout(fc_layer_2, keep_prob)

fc_layer_3 = tf.nn.relu(tf.matmul(fc_layer_2_drop, conv_weights["f3_weights"]) + conv_biases["f3_biases"])
fc_layer_3_drop = tf.nn.dropout(fc_layer_3, keep_prob)

fc_layer_4 = tf.nn.relu(tf.matmul(fc_layer_3_drop, conv_weights["f4_weights"]) + conv_biases["f4_biases"])
fc_layer_4_drop = tf.nn.dropout(fc_layer_4, keep_prob)

y_predicted = tf.identity(tf.matmul(fc_layer_4_drop, conv_weights["f5_weights"]) + conv_biases["f5_biases"])



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
c_layer_5
print(fc_layer_1_drop)

Tensor("dropout/mul_1:0", shape=(?, 1164), dtype=float32)


In [18]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
        print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

(5, 5, 3, 24)
4
5
5
3
24
1800
(5, 5, 24, 36)
4
5
5
24
36
21600
(5, 5, 36, 48)
4
5
5
36
48
43200
(3, 3, 48, 64)
4
3
3
48
64
27648
(3, 3, 64, 64)
4
3
3
64
64
36864
(1152, 1164)
2
1152
1164
1340928
(1164, 100)
2
1164
100
116400
(100, 50)
2
100
50
5000
(50, 10)
2
50
10
500
(10, 1)
2
10
1
10
(24,)
1
24
24
(36,)
1
36
36
(48,)
1
48
48
(64,)
1
64
64
(64,)
1
64
64
(1164,)
1
1164
1164
(100,)
1
100
100
(50,)
1
50
50
(10,)
1
10
10
(1,)
1
1
1
1595511


In [19]:
MYDIR = "NVIDIA"
sess = tf.InteractiveSession()


train_vars = tf.trainable_variables() 

loss = tf.reduce_mean(tf.square(tf.subtract(y_true, y_predicted))) + tf.add_n([tf.nn.l2_loss(w) for w in train_vars]) * 0.001


optimizer = tf.train.AdamOptimizer(learning_rate = 10**-4).minimize(loss)

sess.run(tf.global_variables_initializer())


saver = tf.train.Saver()

epochs = 30

batch_size = 100
epoch_number, train_loss, test_loss,  = [], [], []


In [ ]:
import cv2

for epoch in range(epochs):
    train_avg_loss = 0
    test_avg_loss = 0
    te_loss_old = 10000  #any big number can be given
    
    for i in range(int(len(images)/batch_size)):
        train_batch_x, train_batch_y = loadTrainBatch(batch_size)
        optimizer.run(feed_dict = {x_input: train_batch_x, y_true: train_batch_y, keep_prob: 0.8})
        tr_loss = loss.eval(feed_dict = {x_input: train_batch_x, y_true: train_batch_y, keep_prob: 1.0})
        train_avg_loss += tr_loss / batch_size
    
        test_batch_x, test_batch_y = loadTestBatch(batch_size)
        te_loss_new = loss.eval(feed_dict = {x_input: test_batch_x, y_true: test_batch_y, keep_prob: 1.0})
        test_avg_loss += te_loss_new / batch_size
        
        if te_loss_new < te_loss_old:
            print("Epoch: {}, Train_Loss: {}, Test_Loss: {} *".format(epoch+1, tr_loss, te_loss_new))
        else:
            print("Epoch: {}, Train_Loss: {}, Test_Loss: {}".format(epoch+1, tr_loss, te_loss_new))
        te_loss_old = te_loss_new
        
        if (i+1) % batch_size == 0:
            if not os.path.exists(MYDIR):
                os.makedirs(MYDIR)
            save_path = os.path.join(MYDIR, "model.ckpt")
            saver.save(sess = sess, save_path = save_path)
            print("Model saved at location {} at epoch {}".format(save_path, epoch + 1))
        
    epoch_number.append(epoch)
    train_loss.append(train_avg_loss)
    test_loss.append(test_avg_loss)
    
#creating dataframe and record all the losses and accuracies at each epoch


Epoch: 1, Train_Loss: 6.183895587921143, Test_Loss: 6.481472492218018 *
Epoch: 1, Train_Loss: 6.192596435546875, Test_Loss: 6.890869140625
Epoch: 1, Train_Loss: 21.864961624145508, Test_Loss: 6.462888717651367 *
Epoch: 1, Train_Loss: 7.99490213394165, Test_Loss: 6.196815490722656 *
Epoch: 1, Train_Loss: 7.355810642242432, Test_Loss: 6.150411128997803 *
Epoch: 1, Train_Loss: 9.217510223388672, Test_Loss: 6.160260200500488
Epoch: 1, Train_Loss: 6.216822147369385, Test_Loss: 6.241521835327148
Epoch: 1, Train_Loss: 6.189773082733154, Test_Loss: 7.047809600830078
Epoch: 1, Train_Loss: 7.13718318939209, Test_Loss: 7.692822456359863


In [21]:
import pandas as pd
log_frame = pd.DataFrame(columns = ["Epoch", "Train Loss", "Test Loss"])
log_frame["Epoch"] = epoch_number
log_frame["Train Loss"] = train_loss
log_frame["Test Loss"] = test_loss
log_frame.to_csv(os.path.join(MYDIR, "log.csv"), index = False)

In [22]:
log_frame

,Epoch,Train Loss,Test Loss
0,0,29.240498,28.435796
1,1,27.666242,27.264346
2,2,26.647287,26.185153
3,3,25.661592,25.138753
4,4,24.985177,24.125549


In [24]:
import cv2
import tensorflow.compat.v1 as tf
import scipy

tf.disable_v2_behavior()

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "Saver_adam_2/model.ckpt")

img = cv2.imread('steering_wheel_image.jpg', 0) #here, second parameter '0' specifies that img.shape will return only height and
#width of the image and not the number of channels. It is a colored image so number of channels = 3, which it will not return.
rows, cols = img.shape

images_path = "/Users/yesas/Desktop/Dissertation/driving_dataset/"
Steer_angle = "/Users/yesas/Desktop/Dissertation/driving_dataset/data.txt"

images = []
Angles = []


# In[11]:


with open(Steer_angle) as f:
    for line in f:
        image_name, angle = line.split()
        
        image_path = os.path.join(images_path, image_name)
        images.append(image_path)
        
        angle_radians = float(angle) * (pi / 180) 
        Angles.append(angle_radians)
Angles = np.array(Angles)
print(str(len(images))+" "+str(len(Angles)))




smoothed_angle = 0
i = 0
while(cv2.waitKey(60) != ord("q")):
    full_image = cv2.imread(test_x[i])
    cv2.imshow('Frame Window', full_image)
    image = ((cv2.resize(full_image[-150:], (200, 66)) / 255.0).reshape((1, 66, 200, 3)))
    degrees = sess.run(y_predicted, feed_dict = {x_input: image, keep_prob: 1.0})[0][0] *180 / pi #here, we have converted the
    #predicted degrees from radians to degrees.
    print("Predicted steering angle: " + str(degrees) + " (pred)\t" + str(Angles[i]*180/scipy.pi) + " (actual)")
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2), -smoothed_angle, 1) #this function rotate the image by a given degrees.
    dst = cv2.warpAffine(src = img, M = M, dsize = (cols, rows)) #warpAffine function applies rotation to the image
    cv2.imshow("Steering Wheel", dst)
    i += 1

cv2.destroyAllWindows() 

sess.close()


INFO:tensorflow:Restoring parameters from Saver_adam_2/model.ckpt


C:\Users\yesas\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning:

An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).



45406 45406
Predicted steering angle: 4.293663525492408 (pred)	0.0 (actual)
Predicted steering angle: 4.54276692782512 (pred)	0.0 (actual)
Predicted steering angle: 3.882202684607085 (pred)	0.0 (actual)
Predicted steering angle: 3.8555004869337433 (pred)	0.0 (actual)
Predicted steering angle: 3.7459873682158746 (pred)	0.0 (actual)
Predicted steering angle: 4.363263153134838 (pred)	0.0 (actual)
Predicted steering angle: 4.2858771098385935 (pred)	0.0 (actual)
Predicted steering angle: 4.625291407811329 (pred)	0.0 (actual)
Predicted steering angle: 4.535517108907975 (pred)	0.0 (actual)
Predicted steering angle: 3.632776128949267 (pred)	0.0 (actual)
Predicted steering angle: 3.3948332557203558 (pred)	0.0 (actual)
Predicted steering angle: 3.9263291217399523 (pred)	0.0 (actual)
Predicted steering angle: 3.96695124494178 (pred)	0.0 (actual)
Predicted steering angle: 3.6062557850625767 (pred)	0.0 (actual)
Predicted steering angle: 3.380176523535155 (pred)	0.0 (actual)
Predicted steering angle

Predicted steering angle: 4.892744967122927 (pred)	16.64 (actual)
Predicted steering angle: 4.9800057564117175 (pred)	17.45 (actual)
Predicted steering angle: 5.802409624465913 (pred)	18.05 (actual)
Predicted steering angle: 5.124799363513641 (pred)	18.45 (actual)
Predicted steering angle: 5.30481625616532 (pred)	0.0 (actual)
Predicted steering angle: 4.6440436921776 (pred)	20.27 (actual)
Predicted steering angle: 4.460799962029912 (pred)	20.77 (actual)
Predicted steering angle: 4.793339674603031 (pred)	22.08 (actual)
Predicted steering angle: 4.694084219358052 (pred)	0.0 (actual)
Predicted steering angle: 4.768597314334659 (pred)	24.1 (actual)
Predicted steering angle: 4.163364433582898 (pred)	24.810000000000002 (actual)
Predicted steering angle: 4.1120577600850545 (pred)	0.0 (actual)
Predicted steering angle: 4.559794162539139 (pred)	18.55 (actual)
Predicted steering angle: 4.906667027085529 (pred)	12.81 (actual)
Predicted steering angle: 3.396513055369356 (pred)	9.78 (actual)
Predic

Predicted steering angle: 4.260602421701784 (pred)	7.970000000000001 (actual)
Predicted steering angle: 3.4376807399311384 (pred)	7.970000000000001 (actual)
Predicted steering angle: 3.210267030194625 (pred)	0.0 (actual)
Predicted steering angle: 3.585895418033592 (pred)	8.07 (actual)
Predicted steering angle: 4.305951035809261 (pred)	7.460000000000001 (actual)
Predicted steering angle: 4.198644068193307 (pred)	7.5600000000000005 (actual)
Predicted steering angle: 4.16141996410356 (pred)	6.659999999999999 (actual)
Predicted steering angle: 4.16033823289376 (pred)	5.950000000000001 (actual)
Predicted steering angle: 4.356140119604161 (pred)	5.45 (actual)
Predicted steering angle: 3.2982804209525862 (pred)	5.140000000000001 (actual)
Predicted steering angle: 3.1749711738849378 (pred)	5.140000000000001 (actual)
Predicted steering angle: 3.067825996374948 (pred)	5.65 (actual)
Predicted steering angle: 3.066588024587883 (pred)	8.37 (actual)
Predicted steering angle: 3.1750621007782636 (pred

Predicted steering angle: 4.019797699210638 (pred)	-2.82 (actual)
Predicted steering angle: 3.4500222514311165 (pred)	-2.82 (actual)
Predicted steering angle: 3.5281862959625077 (pred)	-2.82 (actual)
Predicted steering angle: 3.5173117810302013 (pred)	-2.82 (actual)
Predicted steering angle: 3.726686107395065 (pred)	-2.82 (actual)
Predicted steering angle: 3.1709069978853686 (pred)	-2.82 (actual)
Predicted steering angle: 3.5500697951764058 (pred)	-2.62 (actual)
Predicted steering angle: 3.8095704529731695 (pred)	-1.92 (actual)
Predicted steering angle: 3.8983945018082466 (pred)	-1.5100000000000002 (actual)
Predicted steering angle: 4.168621544809692 (pred)	-1.5100000000000002 (actual)
Predicted steering angle: 4.642908600114908 (pred)	-1.5100000000000002 (actual)
Predicted steering angle: 3.9512396754166352 (pred)	-1.5100000000000002 (actual)
Predicted steering angle: 3.644368667518071 (pred)	-1.5100000000000002 (actual)
Predicted steering angle: 3.6675409380509856 (pred)	-1.41 (actua

Predicted steering angle: 3.898424383885865 (pred)	-101.85000000000001 (actual)
Predicted steering angle: 3.5354544710130478 (pred)	-101.85000000000001 (actual)
Predicted steering angle: 3.7074236932751594 (pred)	-101.85000000000001 (actual)
Predicted steering angle: 3.632403456752678 (pred)	-101.85000000000001 (actual)
Predicted steering angle: 3.855445845420383 (pred)	-101.85000000000001 (actual)
Predicted steering angle: 4.343269481886918 (pred)	-101.85000000000001 (actual)
Predicted steering angle: 4.349877689908905 (pred)	-101.45 (actual)
Predicted steering angle: 4.56909602641505 (pred)	-100.93999999999998 (actual)
Predicted steering angle: 4.5086228121642264 (pred)	-99.83 (actual)
Predicted steering angle: 4.6178917516192675 (pred)	-98.62000000000002 (actual)
Predicted steering angle: 4.636775944013883 (pred)	-97.21000000000001 (actual)
Predicted steering angle: 4.581807435347267 (pred)	-96.60999999999999 (actual)
Predicted steering angle: 4.2856021947245 (pred)	-96.609999999999

In [ ]:
sess.close()